In [27]:
import geopandas as gpd
import pandas as pd

header = ['GPST', 'lat_deg', 'long_deg', 'height_m', 'Q', 'ns', 'sdn_m', 'sde_m', 'sdu_m', 'sdne_m', 'sdeu_m', 'sdun_m', 'age_s', 'ratio']
x = pd.read_csv('ublox_complete.pos', delimiter=r"\s\s+", comment='%', header=None, names=header, parse_dates=True, engine = 'python')
